<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Agent/blob/main/OpenAI/00-Naver-Crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Colab: OpenAI Python SDK 설치
!pip -q install --upgrade openai

import os, json, re
from getpass import getpass
from openai import OpenAI

# 안전하게 API Key 입력
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OPENAI_API_KEY: ")

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
MODEL = "gpt-4o-mini"  # Use gpt-4o-mini which supports the search tool

In [7]:
SYSTEM_PROMPT = """\
당신은 신뢰성 높은 한국어 웹 리서처입니다.
반드시 naver.com(하위 도메인 포함: weather.naver.com 등)의 페이지에서만 사실을 인용하세요.
검색어는 한국어로 사용하고, 목표 지역은 '영등포구 여의도동'입니다.
원문 페이지에서 '미세먼지(PM10)'와 '초미세먼지(PM2.5)'의 현재 수치와 등급(좋음/보통/나쁨 등), 단위(㎍/㎥), 관측 시각을 추출하세요.
가장 구체적인 행정동(여의도동) 기준이 우선이며, 불가하면 '영등포구' 수준을 사용하세요.
반드시 아래 JSON 스키마만 출력하세요(추가 텍스트 금지).

{
  "pm10_value": <number or null>,
  "pm10_grade": "<string or null>",
  "pm25_value": <number or null>,
  "pm25_grade": "<string or null>",
  "unit": "<string or null>",
  "observed_at": "<string or null>",
  "source_url": "<string>",
  "source_title": "<string>"
}

가능하면 값을 모두 채우되, 원문에 없는 항목은 null로 두세요.
"""

USER_PROMPT_TEMPLATE = """\
다음을 수행:
1) naver.com 도메인에서만 검색/열람.
2) '{loc} 미세먼지' 또는 '{loc} 초미세먼지' 등 한국어 쿼리 사용.
3) 가장 신뢰되는 네이버 공식/날씨 페이지를 열람하여 현재 수치를 파싱.
4) 지정된 JSON만 반환(불필요한 설명 금지).
"""


def _strip_fence(text: str) -> str:
    """ ```json ... ``` 같은 코드펜스를 제거 """
    m = re.match(r"^\s*```(?:json)?\s*([\s\S]*?)\s*```\s*$", text.strip(), re.IGNORECASE)
    return m.group(1).strip() if m else text.strip()


def fetch_air_quality_from_naver(location: str = "영등포구 여의도동") -> dict:
    """Search Preview 모델로 naver.com에서 미세먼지 정보를 JSON으로 수집"""
    resp = client.responses.create(
        model=MODEL,
        input=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": USER_PROMPT_TEMPLATE.format(loc=location)},
        ],
    )

    raw = resp.output_text or ""
    raw = _strip_fence(raw)

    # JSON 파싱 시도
    try:
        data = json.loads(raw)
    except json.JSONDecodeError:
        data = {
            "pm10_value": None,
            "pm10_grade": None,
            "pm25_value": None,
            "pm25_grade": None,
            "unit": None,
            "observed_at": None,
            "source_url": None,
            "source_title": None,
            "_raw": raw,  # 디버깅용
        }

    # 스키마 키 보정(누락 키가 있더라도 항상 제공)
    for k in [
        "pm10_value", "pm10_grade",
        "pm25_value", "pm25_grade",
        "unit", "observed_at",
        "source_url", "source_title"
    ]:
        data.setdefault(k, None)

    return data


In [8]:
result = fetch_air_quality_from_naver("영등포구 여의도동")
print(json.dumps(result, ensure_ascii=False, indent=2))


{
  "pm10_value": 35,
  "pm10_grade": "보통",
  "pm25_value": 17,
  "pm25_grade": "보통",
  "unit": "㎍/㎥",
  "observed_at": "2023-10-10 10:00",
  "source_url": "https://weather.naver.com/",
  "source_title": "네이버 날씨"
}
